In [6]:
import sys
sys.path.append("..")
from shared_utilities.Betfair import BetfairClient
from shared_utilities import helpers
from azure.cosmos import CosmosClient

In [7]:
delay_api_key = helpers.read_json("../local.settings.json")["Values"]["betfair_delay_api_key"]
username = helpers.read_json("../local.settings.json")["Values"]["betfair_username"]
password = helpers.read_json("../local.settings.json")["Values"]["betfair_password"]
mma_id = 26420387
boxing_id = 6

In [8]:
# initialise this too many times (too many session tokens) and you will get locked out !
bf_client = BetfairClient(delay_api_key, username, password)

In [9]:
data = bf_client.listEvents(mma_id)
data = bf_client.processEvents(data)

In [10]:
conn_string = helpers.read_json("../local.settings.json")["Values"]["cosmosdb_deets"]
storageClient = CosmosClient.from_connection_string(conn_str=conn_string)
database = storageClient.get_database_client("fightstore")
container = database.get_container_client("betfair_mma_events")

In [11]:
existing_items = [i["id"] for i in container.read_all_items()]
# update item or create if not exist.
# TODO: is id always unique???

for i in data:
    if i["id"] in existing_items:
        container.upsert_item(i)
        
    else:
        container.create_item(i)

In [ ]:
data[0]
# container.upsert_item(data[0])

In [ ]:
[i["id"] for i in container.read_all_items()]

In [ ]:
id = 1.223915790

In [ ]:
bf_client.eventStartedChecker(1.223879307)

In [ ]:
bf_client.getMarkets(1.223879307,"True")